In [3]:
import pandas as pd
import numpy as np
from tensorflow import keras
from  sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from statsmodels.tsa.seasonal import seasonal_decompose

label_encoder = LabelEncoder()

InputLayer = keras.layers.InputLayer()
LSTM = keras.layers.LSTM
Dropout = keras.layers.Dropout
Dense = keras.layers.Dense

In [9]:
df = pd.read_csv('/Users/victoroliveira/Desktop/vonix-py-statistic/src/vonixstatisc/data/pandas_data/data_fluency_queue_april_after_24.csv')

df_to_dict = pd.DataFrame()
df = df.loc[(df['handling_time'] > 10) & (df['handling_time'] < 200) ]
df= df.dropna(axis=0)
df


,agent_id,queue_id,week_day,hour,minute,timestamp,handling_time
2,1592,mgmsales,1,21,6,1682370404,114
9,1523,isdragons,1,21,11,1682370675,14
10,1604,mgmsales,1,21,12,1682370735,18
12,1500,iseagles,1,21,13,1682370792,70
13,1583,isdragons,1,21,13,1682370814,11
...,...,...,...,...,...,...,...
9010,1551,mgmsales,2,23,38,1683070689,22
9012,1551,mgmsales,2,23,39,1683070792,22
9016,1605,mgmsdr,3,12,37,1683117431,11
9018,1605,mgmsdr,3,12,38,1683117508,86


In [10]:
df_to_dict['time'] = df['timestamp']
df_to_dict['ht'] = df['handling_time']

In [11]:
dict = df_to_dict.set_index('time').to_dict()['ht']
#dict

In [12]:
df

,agent_id,queue_id,week_day,hour,minute,timestamp,handling_time
2,1592,mgmsales,1,21,6,1682370404,114
9,1523,isdragons,1,21,11,1682370675,14
10,1604,mgmsales,1,21,12,1682370735,18
12,1500,iseagles,1,21,13,1682370792,70
13,1583,isdragons,1,21,13,1682370814,11
...,...,...,...,...,...,...,...
9010,1551,mgmsales,2,23,38,1683070689,22
9012,1551,mgmsales,2,23,39,1683070792,22
9016,1605,mgmsdr,3,12,37,1683117431,11
9018,1605,mgmsdr,3,12,38,1683117508,86


In [13]:
from datetime import datetime
from time import gmtime, strftime
import time
def transform_dict(data: dict, period: int) -> dict:
    """method that receives dict from database query
    {start_at(timestamp): talk_secs(seconds), ...} and create
    another dict based on the period (15, 30, 1hr) chosen
    {last_timestamp: [12,24,56,12, 90, 100], last_timestamp - period:[195,235,201,300], ... interval of three minutes for standard model}
    For standard model the data from the last period( 15 min, 30 min, 1h) is used to construct forecast for the next period so the return_dict begins with the last timestamp to the first
    """
    seconds = period * 60
    
    return_dict = {}

    keys_array = list(data.keys())
    beginning = keys_array[0]
    limit= beginning + seconds
    end = keys_array[-1]
    arr = []
    break_v = 1
    indice = 0
    while beginning < end:
        if break_v == beginning:
            break
        break_v = beginning
        for key, value in data.items():
            if key <= limit:
                arr.append(value)
                continue
        
            return_dict[f"{indice}"] = arr
            indice += 1
            arr = []
            arr.append(value)
            beginning = key
            limit = beginning + seconds
            

    return return_dict

In [14]:
dict_transformed = transform_dict(dict,120)

In [71]:
#dict_transformed

In [16]:
mean_object = {i: np.mean(dict_transformed[i]) for i in dict_transformed.keys()}

In [17]:
mean_object

{'0': 57.710144927536234,
 '1': 17.0,
 '2': 43.51111111111111,
 '3': 60.5,
 '4': 46.65811965811966,
 '5': 54.12337662337662,
 '6': 56.21917808219178,
 '7': 53.04477611940298,
 '8': 55.0,
 '9': 48.34862385321101,
 '10': 53.22119815668203,
 '11': 58.072916666666664,
 '12': 51.0,
 '13': 43.25925925925926,
 '14': 52.424528301886795,
 '15': 48.864,
 '16': 49.568306010928964,
 '17': 50.513698630136986,
 '18': 51.73913043478261,
 '19': 106.0,
 '20': 43.861111111111114,
 '21': 56.467532467532465,
 '22': 49.11538461538461,
 '23': 49.706563706563706,
 '24': 52.642857142857146,
 '25': 54.89473684210526,
 '26': 44.88333333333333,
 '27': 39.43055555555556,
 '28': 43.5,
 '29': 49.791666666666664,
 '30': 39.22222222222222,
 '31': 39.62162162162162,
 '32': 31.77777777777778,
 '33': 37.68627450980392,
 '34': 35.76190476190476,
 '35': 51.04651162790697,
 '36': 49.21052631578947,
 '37': 51.39047619047619,
 '38': 53.12605042016807,
 '39': 55.11764705882353,
 '40': 22.0}

In [18]:
cv_object = {i: np.std(dict_transformed[i]) for i in dict_transformed.keys()}

In [19]:
cv_object

{'0': 47.15472572234951,
 '1': 0.0,
 '2': 29.78636692047198,
 '3': 49.79267622914645,
 '4': 42.528253533399024,
 '5': 45.11663498350446,
 '6': 44.91369092160166,
 '7': 49.57293220134784,
 '8': 41.88465499827263,
 '9': 42.28374579737087,
 '10': 43.361590942576505,
 '11': 51.93811317160443,
 '12': 0.0,
 '13': 37.48737327430209,
 '14': 50.05618505752661,
 '15': 45.7450926767014,
 '16': 43.04356010519238,
 '17': 45.75728646240211,
 '18': 47.875034241154154,
 '19': 0.0,
 '20': 27.224532214915794,
 '21': 45.606613901643925,
 '22': 41.301030833733364,
 '23': 40.862028840149385,
 '24': 43.38846923050414,
 '25': 48.00865572926145,
 '26': 43.265494976430766,
 '27': 31.40807857369435,
 '28': 36.981752257025356,
 '29': 42.679339752000224,
 '30': 37.85678013869028,
 '31': 34.67949885182291,
 '32': 16.75826680098033,
 '33': 39.96002423799458,
 '34': 39.722735197994325,
 '35': 45.59679379200176,
 '36': 37.99664659354257,
 '37': 38.40225569716427,
 '38': 48.46163771251412,
 '39': 45.41881231506088,
 '

In [20]:
call_volume = {i: len(dict_transformed[i]) for i in dict_transformed.keys()}

In [21]:
call_volume

{'0': 69,
 '1': 1,
 '2': 45,
 '3': 66,
 '4': 117,
 '5': 154,
 '6': 73,
 '7': 67,
 '8': 74,
 '9': 109,
 '10': 217,
 '11': 96,
 '12': 1,
 '13': 27,
 '14': 106,
 '15': 125,
 '16': 183,
 '17': 146,
 '18': 23,
 '19': 1,
 '20': 36,
 '21': 77,
 '22': 104,
 '23': 259,
 '24': 182,
 '25': 19,
 '26': 60,
 '27': 72,
 '28': 30,
 '29': 72,
 '30': 27,
 '31': 37,
 '32': 18,
 '33': 51,
 '34': 21,
 '35': 43,
 '36': 57,
 '37': 105,
 '38': 119,
 '39': 68,
 '40': 2}

In [56]:
df_to_ml = pd.DataFrame()
df_to_ml['call_volume'] = call_volume.values()
df_to_ml['cv'] = cv_object.values()
df_to_ml['mean'] = mean_object.values()
df_to_ml

,call_volume,cv,mean
0,69,47.154726,57.710145
1,1,0.000000,17.000000
2,45,29.786367,43.511111
3,66,49.792676,60.500000
4,117,42.528254,46.658120
5,154,45.116635,54.123377
6,73,44.913691,56.219178
7,67,49.572932,53.044776
8,74,41.884655,55.000000
9,109,42.283746,48.348624


In [68]:
np_to_save=df_to_ml.iloc[:, 0:3].values
np_to_save

array([[ 69.        ,  47.15472572,  57.71014493],
       [  1.        ,   0.        ,  17.        ],
       [ 45.        ,  29.78636692,  43.51111111],
       [ 66.        ,  49.79267623,  60.5       ],
       [117.        ,  42.52825353,  46.65811966],
       [154.        ,  45.11663498,  54.12337662],
       [ 73.        ,  44.91369092,  56.21917808],
       [ 67.        ,  49.5729322 ,  53.04477612],
       [ 74.        ,  41.884655  ,  55.        ],
       [109.        ,  42.2837458 ,  48.34862385],
       [217.        ,  43.36159094,  53.22119816],
       [ 96.        ,  51.93811317,  58.07291667],
       [  1.        ,   0.        ,  51.        ],
       [ 27.        ,  37.48737327,  43.25925926],
       [106.        ,  50.05618506,  52.4245283 ],
       [125.        ,  45.74509268,  48.864     ],
       [183.        ,  43.04356011,  49.56830601],
       [146.        ,  45.75728646,  50.51369863],
       [ 23.        ,  47.87503424,  51.73913043],
       [  1.        ,   0.     

In [70]:
np.savetxt("/Users/victoroliveira/Desktop/vonix-py-statistic/jupyter-studies/nn_models/nn_200secs/test_data.txt", np_to_save, delimiter=",")